In [4]:
#import numpy as np
#import geopandas as gpd
#import datetime as dt
#from arcgis.gis import GIS

import ipyleaflet
from ipyleaflet import (Map,DrawControl,GeoJSON,CircleMarker,LayerGroup,Choropleth,GeoData,LayersControl)
import pandas as pd
import branca.colormap as cm
import json
from shapely.geometry import shape, Point

In [5]:
# FUNCTIONS: only run once to initially define the function
# this function takes in a number and returns a colour based on the number

## CHANGE THESE NUMBERS FOR MINIMUM SUFFICIENCY NUMBER OF POINTS IN A WATERSHED!

def getcolour(n):
    if n == 0: return 'grey' # 0 points in the watershed meet the criteria
    if (n>0 and n<4): return 'red'
    if (n>3 and n<7): return 'orange' 
    else: return 'green' # 7+ points in the watershed meet the criteria
    
    
# this function takes in a monitoring location (dict), a parameter name (string), and minimum sample requirements (int)
# this function determines whether a monitoring loc has sufficiency for this parameter (True) or not (False)
def sufficiency_check_singleparam(loc,param,minsamples):
    monitoringparams = loc['par']
    if param in monitoringparams:
        return monitoringparams[param] >= minsamples
    return False


# this function takes in a monitoring location (dict), minimum sample requirements (int), and min param requirements (int)
# this function determines whether a monitoring loc has sufficiency:
# ie. are there at least a minimum number of samples that have a minimum number of parameters measured? (True) or (False)
def sufficiency_check_multiparam(loc,minsamples,minparams):
    monitoringparams = loc['par']
    count = 0
    for p in monitoringparams:
        if monitoringparams[p] >= minsamples:
            count = count + 1
    if count >= minparams:
        return True
    return False

# BUILD A DATABASE
# Function builddatabase: takes in a DataStream dataset (alldata- csv format), and a year (integer) 
# returns a database (dictionary format) of all the locations, each parameter they measure, 
# and how many times each parameter has been measured at each location
# within each location: can call 'lat', 'lon', 'params'
def builddatabase(data):
    
    # filter so the year of the date matches input year:
    datatime = data[pd.DatetimeIndex(data['activity_start_timestamp']).year >= year] # 'ActivityStartDate'

    # Get a list of every location in the dataset
    loclist = list(set(datatime['monitoring_location_name'])) # 'MonitoringLocationName'

    # database to be a list of all locations and their associated lat/lon/parameter counts
    monitoringlocs = []

    # Loop through locations, build database based on information about each location
    for loc in loclist:

        # locinfo (in dictionary form) to store information about each location
        locinfo = {}

        # add name to locinfo database
        locinfo['loc'] = loc

        # filter data so just at that location
        dataloc = datatime[datatime['monitoring_location_name']==loc] # In a normal DS file: 'MonitoringLocationName'

        # extract coordinate information and add to locinfo database
        locinfo['lat'] = dataloc['monitoring_location_latitude'].iloc[0] # In a normal DS file: 'MonitoringLocationLatitude'
        locinfo['lon'] = dataloc['monitoring_location_longitude'].iloc[0] # In a normal DS file: 'MonitoringLocationLongitude'

        # get a list of every parameter measured at this location
        paramlist = list(set(dataloc['characteristic_name'])) # In a normal DS file: 'CharacteristicName'

        # initialize a dictionary for parameter counts
        paramdict = {}

        # loop through all the parameters
        for p in paramlist:

            # filter data so the parameter matches input param:
            data = dataloc[dataloc['characteristic_name']==p] # 'CharacteristicName'

            # count the number of unique dates the param was measured on, then add to dictionary
            paramdict[p] = len(set((data['activity_start_timestamp'])[:9])) # 'ActivityStartDate'

        # add parameter dictionary to the locinfo database
        locinfo['par'] = paramdict  

        # add locinfo to a larger database of all locations
        monitoringlocs.append(locinfo)
        
    return monitoringlocs
    
# function load_wb_data: loads both the sub and subsub watershed boundary data, independent of watershedtype
# appends an ID to the features in the geojson data for easier use when plotting
# returns [subdata subsubdata] -- this is ok since the files are reasonably sized

# this function opens the geojson file
#  -- used mapshaper to compress the size of the watershed boundaries file (maintaining spatial info)
#  -- used qgis to clip to atlantic region only
def load_wb_data():

    #sub data
    with open("./shapefiles/watershed_boundaries_compressed_clipped_sub.geojson") as f:
        sub_data = json.load(f)
        
    # subsub data
    with open("./shapefiles/watershed_boundaries_compressed_clipped.geojson") as f: 
        subsub_data = json.load(f) 


    # add id to the feature in every polygon -- sub
    count = 0
    for feature in sub_data["features"]:
        feature['id'] = count
        count = count + 1
       
    # add id to the feature in every polygon -- subsub
    count = 0
    for feature in subsub_data["features"]:
        feature['id'] = count
        count = count + 1
        
    return sub_data, subsub_data

In [6]:
# input variables
year = 2014
param = 'pH'# 'Temperature, water' # irrelevant if chosen minparams 

watershedtype = 'sub' # this can be 'sub' or 'subsub' depending on which type of ws-boundary you want to analyse

# check sufficiency? if false, this will just plot all points whether or not their number of parameteter are sufficient
checksufficiency = True

# determine number of sufficient locations per watershed
minsamples = 5
minparams = 5

In [7]:
# the following prints some items from the geojson data
# dict(list(wb_gjdata.items())[0:5]) 
#watersheds_gj["features"][0]

In [8]:
# LOAD DATA (DataStream & watershed boundary data) --- just do at the beginning as well!

# testdata = pd.read_csv('MiddleEarth.csv',keep_default_na=False) ### TEST DATASET -- subsection of ACAP data

# DataStream data in 3 files [currently: all the data on DS as of March 2020]
dsdata1 = pd.read_csv('DataStreamData1-2020-03-03.csv',keep_default_na=False,encoding="ISO-8859-1")
#dsdata2 = pd.read_csv('DataStreamData2-2020-03-03.csv',keep_default_na=False,encoding="ISO-8859-1")
#dsdata3 = pd.read_csv('DataStreamData3-2020-03-03.csv',keep_default_na=False,encoding="ISO-8859-1")
#dsdata4 = pd.read_csv('DataStreamData4-2020-03-03.csv',keep_default_na=False,encoding="ISO-8859-1")

#alldata = pd.concat([dsdata1,dsdata2,dsdata3,dsdata4])
alldata = dsdata1

# build database based on this data
monitoringlocs = builddatabase(alldata) # this also can just be done once

# load watershed data
sub_data, subsub_data = load_wb_data()

C:\Users\s1165200\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# use watershedtype (sub or subsub) to determine if we use "WSCSSDA" (subsubdrainage) "WSCSDA" (subdrainage)
if watershedtype == 'sub':
    wstype = 'WSCSDA'
    watersheds_gj = sub_data 
else:
    wstype = 'WSCSSDA'
    watersheds_gj = subsub_data 

# make a list of subsubwatersheds/watersheds (wlist), each with a name and a list of points that are contained within
watersheds = []
wbstyle = []

# circle_marker to be a list of circle markers to plot
circle_marker = []
i = 0


for feature in watersheds_gj['features']:
    polygon = shape(feature['geometry'])
    properties = feature['properties']
    
    watershedinfo = {} # info is in dictionary form
    #watershedinfo['name'] = properties['WSCSSDA']
    watershedinfo['name'] = properties[wstype]

    
    locs_in_wb = []
    
    # create map points and check sufficiency
    
    for loc in monitoringlocs:
        
        # SUFFICIENCY CHECK
        # does the point have required sufficiency????
        if checksufficiency: 
            point = Point(loc['lon'],loc['lat'])
            
            # if the monitoring location is in the ws boundary
            if polygon.contains(point):
                
                # create a circle marker for plotting
                circle_marker.append(CircleMarker())
                circle_marker[i].location = (loc['lat'],loc["lon"])
                circle_marker[i].radius = 1
                
                # if point is sufficient AND in the watershed, add to watershed point list  
                if sufficiency_check_multiparam(loc,minsamples,minparams):
                    locs_in_wb.append(loc)
                    
                    # sufficient points are blue on the map (change later)
                    circle_marker[i].color = "blue"
                    i = i + 1
                    
                else:
                    # insufficient points are red on the map (change later)
                    circle_marker[i].color = "red"
                    i = i + 1
                    
            
    # add the monitoring location to the watershed information
    watershedinfo['locs'] = locs_in_wb
      
    # colour the watershed based on the number of points that are in the watershed
    # note that the number of points will be those that surpassed parameter criteria
    watershedinfo['numpoints'] = len(watershedinfo['locs'])
    watershedinfo['colour'] = getcolour(len(watershedinfo['locs']))
    
    watersheds.append(watershedinfo)
    
    wbstyle.append({'fill':watershedinfo['colour'], 'opacity':1, 'weight':0.5, 'dashArray':'2', 'fillOpacity':0.5})
    


In [11]:
# change style of watershed boundaries for mapping
#wb_map = GeoData(geo_dataframe=wb_gjdata,style = {'fill':watershedinfo['colour'], 'opacity':1, 'weight':0.25, 'dashArray':'2', 'fillOpacity':0.25})

# determine fill colours based on the number of points in each watershed
features = watersheds_gj['features']
ids = [d['id'] for d in features]
nplist = [d['numpoints'] for d in watersheds]
print(nplist)
watershedfill = dict(zip(ids,nplist))

# determines the colour of the watershed based on the number of valid points within it
cmap = cm.StepColormap(colors=['gray','red','yellow','green'],
                       index = [0,1,5,10],
                       vmin = 0,
                       vmax = max(nplist))

wbmap_fill = ipyleaflet.Choropleth(
    geo_data=watersheds_gj,
    choro_data=watershedfill,
    colormap=cmap,
    style={'fillOpacity': 0.4, 'dashArray': '5, 5'})

# make ipyleaflet map with watershed boundaries 
wbmap = Map(center=[50,-62],zoom=5)
wbmap.add_control(LayersControl()) # add layer control
wbmap.add_layer(wbmap_fill)

# add points to map
circle_layer = LayerGroup(layers=circle_marker)
wbmap.add_layer(circle_layer)

# display colormap
display(cmap)

# display map
wbmap



[0, 22, 8, 10, 14, 110, 35, 31, 69, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0]


Map(center=[50, -62], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [17]:
watersheds_gj["features"][0]


{'type': 'Feature',
 'properties': {'PROVCD_1': None,
  'PROVCD_2': None,
  'PROVCD_3': None,
  'PROVCD_4': None,
  'VALDATE': None,
  'EDITION': None,
  'DATASETNAM': '01AB000',
  'VERSION': None,
  'COMPLEVEL': 'NHN-CL2',
  'WSCMDA': '01',
  'WSCSDA': '01A',
  'WSCSSDA': '01AB',
  'FDA': '01AB',
  'OCEAN': 'Atlantic Ocean',
  'WSCMDANAME': 'Maritime Provinces Drainage Area',
  'WSCSDANAME': 'Saint John and Southern Bay of Fundy (N.B.)',
  'WSCSSDANAM': 'Upper Saint John - Big Black'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-69.6739731, 47.2468084],
     [-69.6672169, 47.2493145],
     [-69.6609538, 47.24941190000001],
     [-69.6557632, 47.2546458],
     [-69.6543966, 47.2583238],
     [-69.6474691, 47.2559649],
     [-69.6497894, 47.2501767],
     [-69.6422088, 47.2474566],
     [-69.6384392, 47.2547868],
     [-69.6299307, 47.2604078],
     [-69.6290584, 47.2668782],
     [-69.6205969, 47.269185599999986],
     [-69.6200734, 47.2723509],
     [-69.6243081, 47.27